<a href="https://colab.research.google.com/github/cesar-claros/synergistic/blob/master/socialadsnet_NOTAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
! pip install torch
! pip install gpytorch

In [18]:
#! sudo apt-get install texlive-latex-recommended #1
#! sudo apt-get install dvipng texlive-fonts-recommended #2
#! wget http://mirrors.ctan.org/macros/latex/contrib/type1cm.zip #3
#! unzip type1cm.zip -d /tmp/type1cm #4
#! cd /tmp/type1cm/type1cm/ && sudo latex type1cm.ins  #5
#! sudo mkdir /usr/share/texmf/tex/latex/type1cm #6
#! sudo cp /tmp/type1cm/type1cm/type1cm.sty /usr/share/texmf/tex/latex/type1cm #7
#! sudo texhash #8

In [19]:
# NL - Trust Score for python 3; Change xrange for range on methods fit and get_score
! git clone https://github.com/google/TrustScore
% cd TrustScore/
from sklearn.neighbors import KDTree
from sklearn.neighbors import KNeighborsClassifier
from trustscore import TrustScore

class trust_score(TrustScore): # NL
    def __init__(self,k=10, alpha=0., filtering="none", min_dist=1e-12):
        super().__init__(k,alpha,filtering,min_dist)
    def fit(self, X, y):
      """Initialize trust score precomputations with training data.
      WARNING: assumes that the labels are 0-indexed (i.e.
      0, 1,..., n_labels-1).
      Args:
      X: an array of sample points.
      y: corresponding labels.
      """
      self.n_labels = np.max(y) + 1
      self.kdtrees = [None] * self.n_labels
      if self.filtering == "uncertainty":
        X_filtered, y_filtered = self.filter_by_uncertainty(X, y)
      for label in range(self.n_labels):
        if self.filtering == "none":
          X_to_use = X[np.where(y == label)[0]]
          self.kdtrees[label] = KDTree(X_to_use)
        elif self.filtering == "density":
          X_to_use = self.filter_by_density(X[np.where(y == label)[0]])
          self.kdtrees[label] = KDTree(X_to_use)
        elif self.filtering == "uncertainty":
          X_to_use = X_filtered[np.where(y_filtered == label)[0]]
          self.kdtrees[label] = KDTree(X_to_use)

        if len(X_to_use) == 0:
          print(
              "Filtered too much or missing examples from a label! Please lower "
              "alpha or check data.")

    def get_score(self, X, y_pred):
      """Compute the trust scores.
      Given a set of points, determines the distance to each class.
      Args:
      X: an array of sample points.
      y_pred: The predicted labels for these points.
      Returns:
      The trust score, which is ratio of distance to closest class that was not
      the predicted class to the distance to the predicted class.
      """
      d = np.tile(None, (X.shape[0], self.n_labels))
      for label_idx in range(self.n_labels):
        d[:, label_idx] = self.kdtrees[label_idx].query(X, k=2)[0][:, -1]

      sorted_d = np.sort(d, axis=1)
      d_to_pred = d[range(d.shape[0]), y_pred]
      d_to_closest_not_pred = np.where(sorted_d[:, 0] != d_to_pred,
                                      sorted_d[:, 0], sorted_d[:, 1])
      return d_to_closest_not_pred / (d_to_pred + self.min_dist)
% cd .. 
# help(trust_score)

Cloning into 'TrustScore'...
remote: Enumerating objects: 14, done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 14
Unpacking objects: 100% (14/14), done.
/content/synergistic/TrustScore
/content/synergistic


In [20]:
! git clone https://github.com/cesar-claros/synergistic
% cd synergistic/

Cloning into 'synergistic'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 185 (delta 80), reused 10 (delta 0), pack-reused 0
Receiving objects: 100% (185/185), 11.46 MiB | 7.83 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/synergistic/synergistic


In [21]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [22]:
#%%
# Imports
import io #Used as buffer
import sys
import matplotlib
# matplotlib.use('qt5Agg')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import auxfunc.funcs as sgn
import seaborn as sns
import torch
import tensorflow as tf
from scipy.stats import entropy, spearmanr
from sklearn import model_selection, svm, ensemble, linear_model, pipeline, decomposition,\
     tree, neighbors, discriminant_analysis, gaussian_process, preprocessing
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern
plt.style.use(['ggplot','style/style.mplstyle'])
import os
import random

In [23]:
#%%
# MODELS
# ====================
# Grid search for parameters and classifiers

models = { 
        'predictor':
            [
            svm.SVC(),
            gaussian_process.GaussianProcessClassifier(),
            linear_model.LinearRegression(),
            linear_model.Lasso(),
            svm.SVR()],
        'name':
            [
            'SVM',
            'GPClassifier',
            'LinReg',
            'Lasso',
            'SVR']}
parameters = [
            {'SVM__kernel':['poly'],'SVM__degree':[3,4,5]},
            {'GPClassifier__kernel':[]},
            {},
            {'Lasso__alpha':np.linspace(0.01,1,10)},
            {'SVR__kernel':['linear'], 'SVR__C':np.logspace(-1, 1, 10), 'SVR__epsilon':np.logspace(-2, 2, 10)} ]


In [24]:
#%%
# Signaling function fitting and evaluation
def signalingFunction(X_train, y_train, y_train_pred_th, X_val, y_val, y_val_pred_th, X_test, y_test, y_test_pred_th, kernel='exponential', norm='l01'):
    # X_train, X_val should be scaled
    # Fit signaling function 
    exp = sgn.signaling(norm=norm) # idx = [train,test,val]
    exp.fit(X_train, y_train, y_train_pred_th, kernel=kernel, n_iter=500, lr=0.01)
    table_val = exp.evaluate(X_val, y_val, y_val_pred_th, rule_grid=np.linspace(0,3,30, endpoint=False), rho_grid=[0.1, 0.15])
    table_test = exp.test(X_test, y_test, y_test_pred_th, table_val['rule'].to_numpy(), table_val['eta'].to_numpy())
    table = pd.concat([table_val,table_test],axis=1)
    return table, exp

In [25]:
#%%
# Signaling function fitting and evaluation
def patchFunction(X_train, y_train, y_train_pred_soft, y_train_pred_th, 
                  X_val, y_val, y_val_pred_soft, y_val_pred_th, 
                  X_test, y_test, y_test_pred_soft, y_test_pred_th, 
                  kernel='e*e', ex_dim=1, norm='res'):
    # X_train, X_val should be scaled
    # Add predictions
    X_train_GP = np.concatenate((X_train, y_train_pred_soft), axis=1)
    X_val_GP = np.concatenate((X_val, y_val_pred_soft), axis=1)
    X_test_GP = np.concatenate((X_test, y_test_pred_soft), axis=1)
    # Scale features
    scaleX_GP = preprocessing.StandardScaler().fit(np.concatenate((X_train_GP, X_val_GP), axis=0))
    X_train_GP = scaleX_GP.transform(X_train_GP)
    X_val_GP = scaleX_GP.transform(X_val_GP)
    X_test_GP = scaleX_GP.transform(X_test_GP)
    # Fit signaling function 
    patch = sgn.patching(norm=norm) # idx = [train,test,val]
    patch.fit(X_train_GP, y_train, y_train_pred_soft[:,1], kernel=kernel, n_iter=500, lr=0.01, ex_dim=ex_dim)
    table_patch_only = patch.apply(X_val_GP, y_val, y_val_pred_soft, y_val_pred_th, 
                    X_test_GP, y_test, y_test_pred_soft, y_test_pred_th)
    table_patch_val = patch.evaluate(y_val, y_val_pred_soft, y_val_pred_th)
    table_patch_test = patch.test(y_test, y_test_pred_soft, y_test_pred_th, table_patch_val['thresh'].to_numpy())
    table_patch = pd.concat([table_patch_val, table_patch_test], axis=1)
    return table_patch_only, table_patch

In [26]:
#%%
# Initialize model
def init_model(input_dim, models, parameters, clf):
    
    if clf=='svm':
      i = 0
    elif clf=='gpc':
      i = 1
      kernel = 1.0 * RBF(length_scale=1.0*np.ones(input_dim)) # RFB = Squared Exponental Kernel. As length scale = [1,...,1] (vector), it is the anisotropic variant of the kernel (diferentes sigma para c dimension, también para GP regression)
      parameters[i]['GPClassifier__kernel'].append(kernel) # Se añade a la lista de parmámetros del 'GPClassifier' el kernel recién definido
      
    scaler = preprocessing.StandardScaler() # Iniciación de estandarización de datos
    steps = [('scaler', scaler), (models['name'][i], models['predictor'][i])] # Definicion de pipeline 1)Scaler, 2) (Nombre_Modelo,Llamada_Modelo)
    ppline = pipeline.Pipeline(steps) # define the pipeline object.

    clf = model_selection.GridSearchCV(ppline, param_grid=parameters[i], cv=5, iid=False) # Iniciación de Búsqueda de parámetros de optimización para el modelo seleccionado, i =0 o i=1
    return clf

In [27]:
#%%
# Soft and thresholded output predictions
def pred_output(model, X):
    if hasattr(model, "decision_function"): # Si tiene el atributo "decision_function" (SVM lo tienen y GPC no)
      y_pred_soft = model.best_estimator_.decision_function(X)[:,None] # Evalua la decision function en X
      y_pred_th = model.best_estimator_.predict(X)          # Para el modelo de validación cruzada con menor error de CV (?check) predice de manera discreta el resultado purchase/no purchase
    else:
      y_pred_soft = model.best_estimator_.predict_proba(X)  # Para el modelo de validación cruzada con menor error de CV (?check) predice de manera continua el resultado purchase/no purchase
      y_pred_th = model.best_estimator_.predict(X)          # Para el modelo de validación cruzada con menor error de CV (?check) predice de manera discreta el resultado purchase/no purchase
    return y_pred_soft, y_pred_th

In [28]:
#%%
# Jaccard similarity index
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [29]:
#%%
# Baseline comparison
def baselineCriteria(y_val, y_val_pred_soft, y_val_pred_th, y_test, y_test_pred_soft, y_test_pred_th, table, exp, clf,trust_val,trust_test): #NL
      if clf=='svm':           # SVM es un modelo en R. g(x) - Margin
          direction = 'closer'
          crit_val = np.abs(y_val_pred_soft.ravel())
          crit_test = np.abs(y_test_pred_soft.ravel())
      else:
          direction = 'further' # Los demás son modelos en [0,1] - entropy
          p_val = np.concatenate((y_val_pred_soft,1-y_val_pred_soft),axis=1)
          crit_val = entropy(p_val, axis=1, base=2)
          p_test = np.concatenate((y_test_pred_soft,1-y_test_pred_soft),axis=1)
          crit_test = entropy(p_test, axis=1, base=2)
      # crit_val  - valor de la signailing function alternativa correspondiente (en val)
      # crit_test - valor de la signailing function alternativa correspondiente (en test)
      critFunc   = sgn.critEvaluation(norm='l01',direction=direction)
      d_val      = critFunc.evaluate(y_val, y_val_pred_th, crit_val, rho_grid=[0.1, 0.15]) # same rho grid as in 'signalingFunction'
      d_test     = critFunc.test(y_test, y_test_pred_th, crit_test, d_val['thresh'].to_numpy())
      # TrustScore
      critFuncSc = sgn.critEvaluation(norm='l01',direction='closer') #NL
      score_val  = critFuncSc.evaluate(y_val, y_val_pred_th, trust_val, rho_grid=[0.1, 0.15]) # NL
      score_test = critFuncSc.test(y_test, y_test_pred_th, trust_test, score_val['thresh'].to_numpy()) #NL 

      crit_table = pd.concat([d_val,d_test],axis=1)
      score_table= pd.concat([score_val,score_test],axis=1) #NL

      gamma = table['rule'].to_numpy().reshape(-1,1) #Trae la columna de best rules en val
      f_test = exp.gpr_mean_test + gamma*np.sqrt(exp.gpr_var_test)#Trae
      eta = table['eta'].to_numpy().reshape(-1,1)
      theta = crit_table['thresh'].to_numpy().reshape(-1,1)
      if direction == 'closer':
        f_mask, f_idx = np.nonzero(f_test>eta)
      else:
        f_mask, f_idx = np.nonzero(f_test<eta)
      crit_mask, crit_idx = np.nonzero(crit_test.reshape(1,-1)<theta)
      print(list(np.unique(f_mask)))
      print(list(np.unique(crit_mask)))
      print(f_test.shape[0])
      shared = set(list(np.unique(f_mask))).intersection(set(list(np.unique(crit_mask))))
      J = [jaccard_similarity(crit_idx[crit_mask==i],f_idx[f_mask==i]) if i in shared else np.nan for i in range(f_test.shape[0])]
      # if (list(np.unique(f_mask))==list(np.unique(crit_mask))):
      #   J = [jaccard_similarity(crit_idx[crit_mask==i],f_idx[f_mask==i]) for i in np.unique(f_mask)]
      # else:
      #   shared = set(a).intersection(set(b))
      #   union = set(a).union(set(b))
      #   J = [jaccard_similarity(crit_idx[crit_mask==i],f_idx[f_mask==i]) if i in shared else np.nan  for i in union]
      crit_table['jaccard']=J
      Sp = [spearmanr(f_test[i,:],crit_test)[0] for i in range(f_test.shape[0])]
      crit_table['spearman'] = Sp
      crit_table['gamma'] = gamma
      return crit_table

In [30]:
# For reproducibility
os.environ['TF_DETERMINISTIC_OPS'] = '1'
SEED = 123
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [31]:
#%%
# INITIALIZATION
# ==============
# EXPERIMENT SETUP
# ================
# Load data
# -------------
df = pd.read_table('datasets/Social_Network_Ads.csv')
Data_X = df.iloc[:,[2,3]]
Data_y = df.iloc[:,4]
Data_X = Data_X.to_numpy()
Data_y = Data_y.to_numpy()

In [32]:
# NL. Adición de trust score
# Assign labels
report_table = []
report_criteria = []
report_table_patch_only = []
report_table_patch = []
kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
# kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=SEED)
clf = 'gpc'
addPredictions = True
evaluatePatch = False
accuracy = 0
for sample, test in kf.split(Data_X, Data_y):
# for sample, test in kf.split(Data_X):
    X = Data_X[sample]    # Toma 80% de los 400 datos = 320 sujetos para las 2 covariables. Esta matriz se usa para entrenar el modelo para el trust score
    y = Data_y[sample]    # Toma 80% de los 400 datos = 320 respuestas en {0,1}. Este vector se usa para entrenar el modelo para el trust score
    X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.20, random_state=SEED) # Los 320 datos del fold actual son divididos en training (80% - 256) y testing-validation (20% - 64)
    X_test = Data_X[test] # Toma 20% de los 400 datos = 80
    y_test = Data_y[test] # Toma 20% de los 400 datos = 80

    # TRAINING MODEL - Modelo de entrenamiento de h. En training data
    model = init_model(input_dim=X.shape[1], models=models, parameters=parameters, clf=clf) # Determinación de objeto de validación cruzada para seleccion de modelo. En este caso, seleccion **asintropic** de Modelo clf = GPC (definido arriba)
    model.fit(X_train, y_train) # Actual fit del modelo. Ejecución del objeto clf = model_selection.GridSearchCV(...)

    y_train_pred_soft, y_train_pred_th = pred_output(model, X_train) # train - Predicción del modelo entrenado en 'train' - th {0,1}, soft [0,1] o [0,inf]
    print('accuracy(Train)={}'.format(np.sum(y_train==y_train_pred_th)/np.size(y_train)))
    y_val_pred_soft, y_val_pred_th = pred_output(model, X_val)       # val   - Predicción del modelo entrenado en 'train' - th {0,1}, soft [0,1] o [0,inf]
    y_test_pred_soft, y_test_pred_th = pred_output(model, X_test)    # test  - Predicción del modelo entrenado en 'train' - th {0,1}, soft [0,1] o [0,inf]

    # PATCH MODEL
    if evaluatePatch:
        table_patch_only, table_patch = patchFunction(X_train, y_train, y_train_pred_soft, y_train_pred_th, 
                                                               X_val, y_val, y_val_pred_soft, y_val_pred_th, 
                                                               X_test, y_test, y_test_pred_soft, y_test_pred_th,
                                                               kernel='RBF+RBF', ex_dim=y_train_pred_soft.shape[1])
        report_table_patch_only.append(table_patch_only)
        report_table_patch.append(table_patch)

    X_train_GP = X_train
    X_val_GP = X_val
    X_test_GP = X_test
    if addPredictions: # Se añaden las predicciones del modelo en la matriz diseño para configurar \Delta^{**}
        # Add predictions
        X_train_GP = np.concatenate((X_train_GP, y_train_pred_soft), axis=1) # matriz diseño train + pronostico continuo
        X_val_GP = np.concatenate((X_val_GP, y_val_pred_soft), axis=1)       # matriz diseño val   + pronostico continuo
        X_test_GP = np.concatenate((X_test_GP, y_test_pred_soft), axis=1)    # matriz diseño test  + pronostico continuo
    scaleX_GP = preprocessing.StandardScaler().fit(X_train_GP) # NL - Xq concatenate Mean and std dev of [train,val] data to standarize the other data
    X_train_GP = scaleX_GP.transform(X_train_GP)
    X_val_GP = scaleX_GP.transform(X_val_GP)
    X_test_GP = scaleX_GP.transform(X_test_GP)
    
    table, exp = signalingFunction(X_train_GP, y_train, y_train_pred_th, X_val_GP, y_val, y_val_pred_th, X_test_GP, y_test, y_test_pred_th)
    report_table.append(table)
    
    # Baseline for comparison
    # Trust Score fitted on train+val data to evaluate loss reduction on test data
    trust_model = trust_score()
    trust_model.fit(X=X_train_GP,y=y_train) #NL
    trust_val  = trust_model.get_score(X_val_GP, y_val_pred_th) #NL
    trust_test = trust_model.get_score(X_test_GP, y_test_pred_th) #NL
                                
    crit_table = baselineCriteria(y_val, y_val_pred_soft, y_val_pred_th, y_test, y_test_pred_soft, y_test_pred_th, table, exp, clf,trust_val,trust_test)
    report_criteria.append(crit_table)

    if accuracy < model.best_estimator_.score(X_val,y_val):
        accuracy = model.best_estimator_.score(X_val,y_val)
        classifier = model.best_estimator_
        X_test_surface_plot = X_test
        y_test_surface_plot = y_test
        X_train_surface_plot = X_train
        y_train_surface_plot = y_train
        X_val_surface_plot = X_val
        y_val_surface_plot = y_val
        scaler_surface_plot = scaleX_GP
        exp_surface_plot = exp
        table_surface_plot = table

    del(model)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


accuracy(Train)=0.9140625
lr=0.01, n_iterations=500
Iter 491/500 - Loss: 0.033  noise: 0.045
Iter 492/500 - Loss: 0.033  noise: 0.045
Iter 493/500 - Loss: 0.033  noise: 0.045
Iter 494/500 - Loss: 0.033  noise: 0.045
Iter 495/500 - Loss: 0.032  noise: 0.045
Iter 496/500 - Loss: 0.032  noise: 0.045
Iter 497/500 - Loss: 0.033  noise: 0.044
Iter 498/500 - Loss: 0.032  noise: 0.044
Iter 499/500 - Loss: 0.032  noise: 0.044
Iter 500/500 - Loss: 0.032  noise: 0.044
[0, 1]
[0, 1]
2


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


accuracy(Train)=0.921875
lr=0.01, n_iterations=500
Iter 491/500 - Loss: 0.001  noise: 0.041
Iter 492/500 - Loss: 0.001  noise: 0.041
Iter 493/500 - Loss: 0.001  noise: 0.041
Iter 494/500 - Loss: 0.001  noise: 0.041
Iter 495/500 - Loss: 0.001  noise: 0.041
Iter 496/500 - Loss: 0.000  noise: 0.040
Iter 497/500 - Loss: 0.000  noise: 0.040
Iter 498/500 - Loss: 0.000  noise: 0.040
Iter 499/500 - Loss: 0.000  noise: 0.040
Iter 500/500 - Loss: -0.000  noise: 0.040
[0, 1]
[0, 1]
2


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


accuracy(Train)=0.9140625
lr=0.01, n_iterations=500
Iter 491/500 - Loss: -0.056  noise: 0.032
Iter 492/500 - Loss: -0.056  noise: 0.032
Iter 493/500 - Loss: -0.056  noise: 0.032
Iter 494/500 - Loss: -0.056  noise: 0.032
Iter 495/500 - Loss: -0.056  noise: 0.032
Iter 496/500 - Loss: -0.056  noise: 0.032
Iter 497/500 - Loss: -0.056  noise: 0.032
Iter 498/500 - Loss: -0.056  noise: 0.032
Iter 499/500 - Loss: -0.057  noise: 0.032
Iter 500/500 - Loss: -0.056  noise: 0.032
[0, 1]
[0, 1]
2


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


accuracy(Train)=0.921875
lr=0.01, n_iterations=500
Iter 491/500 - Loss: -0.014  noise: 0.039
Iter 492/500 - Loss: -0.014  noise: 0.038
Iter 493/500 - Loss: -0.015  noise: 0.038
Iter 494/500 - Loss: -0.016  noise: 0.038
Iter 495/500 - Loss: -0.016  noise: 0.038
Iter 496/500 - Loss: -0.017  noise: 0.038
Iter 497/500 - Loss: -0.018  noise: 0.038
Iter 498/500 - Loss: -0.018  noise: 0.038
Iter 499/500 - Loss: -0.019  noise: 0.037
Iter 500/500 - Loss: -0.020  noise: 0.037
[0, 1]
[0, 1]
2


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


accuracy(Train)=0.90234375
lr=0.01, n_iterations=500
Iter 491/500 - Loss: 0.130  noise: 0.054
Iter 492/500 - Loss: 0.129  noise: 0.054
Iter 493/500 - Loss: 0.129  noise: 0.054
Iter 494/500 - Loss: 0.129  noise: 0.054
Iter 495/500 - Loss: 0.129  noise: 0.054
Iter 496/500 - Loss: 0.129  noise: 0.054
Iter 497/500 - Loss: 0.129  noise: 0.054
Iter 498/500 - Loss: 0.129  noise: 0.054
Iter 499/500 - Loss: 0.129  noise: 0.053
Iter 500/500 - Loss: 0.129  noise: 0.053
[0, 1]
[0, 1]
2


In [ ]:
logs_base_dir = "runs/"
%tensorboard --logdir {logs_base_dir}

In [ ]:
report_table_concat = pd.concat(report_table)
report_table_concat

In [ ]:
report_criteria_concat = pd.concat(report_criteria)
report_criteria_concat

In [ ]:
# p-value column
p_value_col = report_table_concat['p_value'] #Add
cols_CQT = ['rho_user','corrected_test','queries_test','total_wrong_test','loss_query_test']
cols_rholoss = ['rho_user','rho_hat_test','%loss_red_test']

# Dataframes for f(x)
df_fx_CQT = pd.DataFrame(report_table_concat[cols_CQT])
df_fx_rholoss = pd.DataFrame(report_table_concat[cols_rholoss])
# results_fx_CQT = df_fx_CQT.loc[p_value_col <= 0.05].copy()
results_fx_rholoss = df_fx_rholoss.loc[p_value_col <= 0.05].copy()
results_fx_CQT = df_fx_CQT.copy()
# results_fx_rholoss = df_fx_rholoss.copy()

results_fx_rholoss_bri = results_fx_rholoss.groupby(results_fx_rholoss.index)
fx_rholoss_median = results_fx_rholoss_bri.median()
fx_rholoss_q1 = results_fx_rholoss_bri.quantile(q=0)
fx_rholoss_q3 = results_fx_rholoss_bri.quantile(q=1)
results_fx_CQT_bri = results_fx_CQT.groupby(results_fx_CQT.index)
fx_CQT_median = results_fx_CQT_bri.median()
fx_CQT_q1 = results_fx_CQT_bri.quantile(q=0)
fx_CQT_q3 = results_fx_CQT_bri.quantile(q=1)

# Dataframes for g(x)
df_gx_CQT = pd.DataFrame(report_criteria_concat[cols_CQT])
df_gx_rholoss = pd.DataFrame(report_criteria_concat[cols_rholoss])
# results_gx_CQT = df_gx_CQT.loc[p_value_col <= 0.05].copy()
results_gx_rholoss = df_gx_rholoss.loc[p_value_col <= 0.05].copy()
results_gx_CQT = df_gx_CQT.copy()
# results_gx_rholoss = df_gx_rholoss.copy()

results_gx_rholoss_bri = results_gx_rholoss.groupby(results_gx_rholoss.index)
gx_rholoss_median = results_gx_rholoss_bri.median()
gx_rholoss_q1 = results_gx_rholoss_bri.quantile(q=0)
gx_rholoss_q3 = results_gx_rholoss_bri.quantile(q=1)
results_gx_CQT_bri = results_gx_CQT.groupby(results_gx_CQT.index)
gx_CQT_median = results_gx_CQT_bri.median()
gx_CQT_q1 = results_gx_CQT_bri.quantile(q=0)
gx_CQT_q3 = results_gx_CQT_bri.quantile(q=1)

# Signaling function statistics (median(q1-q3)) LaTex
output_test = io.StringIO()
# numRows = fx_median.shape[0]
# numCols = fx_median.shape[1]
output_test.write("results_test (dataset|method|(Q,C,T)|\hat{rho_test}|%loss_red_test\n")
output_test.write("----------\n")

for rho in [0.10,0.15]:
  # output_test.write("rho={:.2f}\\\\\n".format(rho))

  fx_CQT_filtered = results_fx_CQT.loc[results_fx_CQT['rho_user']==rho]
  gx_CQT_filtered = results_gx_CQT.loc[results_gx_CQT['rho_user']==rho]
  p_value_filtered = p_value_col[results_fx_CQT['rho_user']==rho]
  # print(p_value_filtered)
  n_folds = fx_CQT_filtered.shape[0]

  row_fx = [' ',' ',r'{:.2f}'.format(rho),r'$f(x)$']
  row_fx_CQT = []
  row_gx = [' ',' ',' ',r'$g(x)$']
  row_gx_CQT = []
  for i in range(n_folds):
    if p_value_filtered.iloc[i] <= 0.05:
      row_fx_CQT.append('({Q:.0f}/{C:.0f}/{T:.0f})'.format(
                                                  Q=fx_CQT_filtered['queries_test'].iloc[i],\
                                                  C=fx_CQT_filtered['corrected_test'].iloc[i],\
                                                  T=fx_CQT_filtered['total_wrong_test'].iloc[i]))
      row_gx_CQT.append('({Q:.0f}/{C:.0f}/{T:.0f})'.format(
                                                  Q=gx_CQT_filtered['queries_test'].iloc[i],\
                                                  C=gx_CQT_filtered['corrected_test'].iloc[i],\
                                                  T=gx_CQT_filtered['total_wrong_test'].iloc[i]))
    else:
      row_fx_CQT.append('\\textit{{({Q:.0f}/{C:.0f}/{T:.0f})}}'.format(
                                                  Q=fx_CQT_filtered['queries_test'].iloc[i],\
                                                  C=fx_CQT_filtered['corrected_test'].iloc[i],\
                                                  T=fx_CQT_filtered['total_wrong_test'].iloc[i]))
      row_gx_CQT.append('\\textit{{({Q:.0f}/{C:.0f}/{T:.0f})}}'.format(
                                                  Q=gx_CQT_filtered['queries_test'].iloc[i],\
                                                  C=gx_CQT_filtered['corrected_test'].iloc[i],\
                                                  T=gx_CQT_filtered['total_wrong_test'].iloc[i]))
  row_fx.append("{:s}".format(", ".join(row_fx_CQT)))
  row_gx.append("{:s}".format(", ".join(row_gx_CQT)))

  row_fx_rho = [r'{:.2f}'.format(val1) for val1 in fx_rholoss_median.loc[fx_rholoss_median['rho_user']==rho,'rho_hat_test']]
  row_fx_loss = [r'{:.1f}'.format(val1) for val1 in fx_rholoss_median.loc[fx_rholoss_median['rho_user']==rho,'%loss_red_test']]
  row_fx_lq = [r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) for val1,val2,val3 in zip(fx_CQT_median.loc[fx_CQT_median['rho_user']==rho,'loss_query_test'],\
                                                      fx_CQT_q3.loc[fx_CQT_q3['rho_user']==rho,'loss_query_test'],\
                                                      fx_CQT_q1.loc[fx_CQT_q1['rho_user']==rho,'loss_query_test'])]
  row_fx = row_fx + row_fx_rho + row_fx_loss + row_fx_lq
  output_test.write("{:s}\\\\\n".format(" & ".join(row_fx)))

  row_gx_rho = [r'{:.2f}'.format(val1) for val1 in gx_rholoss_median.loc[gx_rholoss_median['rho_user']==rho,'rho_hat_test']]
  row_gx_loss = [r'{:.1f}'.format(val1) for val1 in gx_rholoss_median.loc[gx_rholoss_median['rho_user']==rho,'%loss_red_test']]
  row_gx_lq = [r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) for val1,val2,val3 in zip(gx_CQT_median.loc[gx_CQT_median['rho_user']==rho,'loss_query_test'],\
                                                      gx_CQT_q3.loc[gx_CQT_q3['rho_user']==rho,'loss_query_test'],\
                                                      gx_CQT_q1.loc[gx_CQT_q1['rho_user']==rho,'loss_query_test'])]
  row_gx = row_gx + row_gx_rho + row_gx_loss + row_gx_lq
  output_test.write("{:s}\\\\\n".format(" & ".join(row_gx)))
  output_test.write("\\cline{3-8}\n")


print(output_test.getvalue())

In [ ]:
# p-value column
p_value_col = report_table_concat['p_value'] #Add
cols_CQT = ['rho_user','corrected_test','queries_test','total_wrong_test','loss_query_test']
cols_rholoss = ['rho_user','rho_hat_test','%loss_red_test']

# Dataframes for f(x)
df_fx_CQT = pd.DataFrame(report_table_concat[cols_CQT])
df_fx_rholoss = pd.DataFrame(report_table_concat[cols_rholoss])
# results_fx_CQT = df_fx_CQT.loc[p_value_col <= 0.05].copy()
results_fx_rholoss = df_fx_rholoss.loc[p_value_col <= 0.05].copy()
results_fx_CQT = df_fx_CQT.copy()
# results_fx_rholoss = df_fx_rholoss.copy()

results_fx_rholoss_bri = results_fx_rholoss.groupby(results_fx_rholoss.index)
fx_rholoss_median = results_fx_rholoss_bri.median()
fx_rholoss_q1 = results_fx_rholoss_bri.quantile(q=0)
fx_rholoss_q3 = results_fx_rholoss_bri.quantile(q=1)
results_fx_CQT_bri = results_fx_CQT.groupby(results_fx_CQT.index)
fx_CQT_median = results_fx_CQT_bri.median()
fx_CQT_q1 = results_fx_CQT_bri.quantile(q=0)
fx_CQT_q3 = results_fx_CQT_bri.quantile(q=1)

# Dataframes for g(x)
df_gx_CQT = pd.DataFrame(report_criteria_concat[cols_CQT])
df_gx_rholoss = pd.DataFrame(report_criteria_concat[cols_rholoss])
# results_gx_CQT = df_gx_CQT.loc[p_value_col <= 0.05].copy()
results_gx_rholoss = df_gx_rholoss.loc[p_value_col <= 0.05].copy()
results_gx_CQT = df_gx_CQT.copy()
# results_gx_rholoss = df_gx_rholoss.copy()

results_gx_rholoss_bri = results_gx_rholoss.groupby(results_gx_rholoss.index)
gx_rholoss_median = results_gx_rholoss_bri.median()
gx_rholoss_q1 = results_gx_rholoss_bri.quantile(q=0)
gx_rholoss_q3 = results_gx_rholoss_bri.quantile(q=1)
results_gx_CQT_bri = results_gx_CQT.groupby(results_gx_CQT.index)
gx_CQT_median = results_gx_CQT_bri.median()
gx_CQT_q1 = results_gx_CQT_bri.quantile(q=0)
gx_CQT_q3 = results_gx_CQT_bri.quantile(q=1)

# Signaling function statistics (median(q1-q3)) LaTex
output_test = io.StringIO()
# numRows = fx_median.shape[0]
# numCols = fx_median.shape[1]
output_test.write("results_test (dataset|method|(Q,C,T)|\hat{rho_test}|%loss_red_test\n")
output_test.write("----------\n")

for rho in [0.10,0.15]:
  # output_test.write("rho={:.2f}\\\\\n".format(rho))

  fx_CQT_filtered = results_fx_CQT.loc[results_fx_CQT['rho_user']==rho]
  gx_CQT_filtered = results_gx_CQT.loc[results_gx_CQT['rho_user']==rho]
  p_value_filtered = p_value_col[results_fx_CQT['rho_user']==rho]
  # print(p_value_filtered)
  n_folds = fx_CQT_filtered.shape[0]

  row_fx = [' ',' ',r'{:.2f}'.format(rho),r'$f(x)$']
  row_gx = [' ',' ',' ',r'$g(x)$']
  # for i in range(n_folds):
  #   if p_value_filtered.iloc[i] <= 0.05:
  #     row_fx_CQT.append('({Q:.0f}/{C:.0f}/{T:.0f})'.format(
  #                                                 Q=fx_CQT_filtered['queries_test'].iloc[i],\
  #                                                 C=fx_CQT_filtered['corrected_test'].iloc[i],\
  #                                                 T=fx_CQT_filtered['total_wrong_test'].iloc[i]))
  #     row_gx_CQT.append('({Q:.0f}/{C:.0f}/{T:.0f})'.format(
  #                                                 Q=gx_CQT_filtered['queries_test'].iloc[i],\
  #                                                 C=gx_CQT_filtered['corrected_test'].iloc[i],\
  #                                                 T=gx_CQT_filtered['total_wrong_test'].iloc[i]))
  #   else:
  #     row_fx_CQT.append('\\textit{{({Q:.0f}/{C:.0f}/{T:.0f})}}'.format(
  #                                                 Q=fx_CQT_filtered['queries_test'].iloc[i],\
  #                                                 C=fx_CQT_filtered['corrected_test'].iloc[i],\
  #                                                 T=fx_CQT_filtered['total_wrong_test'].iloc[i]))
  #     row_gx_CQT.append('\\textit{{({Q:.0f}/{C:.0f}/{T:.0f})}}'.format(
  #                                                 Q=gx_CQT_filtered['queries_test'].iloc[i],\
  #                                                 C=gx_CQT_filtered['corrected_test'].iloc[i],\
  #                                                 T=gx_CQT_filtered['total_wrong_test'].iloc[i]))
  # row_fx.append("{:s}".format(", ".join(row_fx_CQT)))
  # row_gx.append("{:s}".format(", ".join(row_gx_CQT)))

  row_fx_CQT = [r'({:.2f},{:.2f},{:.2f})'.format(val1,val2,val3) for val1,val2,val3 in zip(fx_CQT_median.loc[fx_CQT_median['rho_user']==rho,'queries_test'],\
                                                      fx_CQT_median.loc[fx_CQT_q3['rho_user']==rho,'corrected_test'],\
                                                      fx_CQT_median.loc[fx_CQT_q1['rho_user']==rho,'total_wrong_test'])]
  row_fx_rho = [r'{:.2f}'.format(val1) for val1 in fx_rholoss_median.loc[fx_rholoss_median['rho_user']==rho,'rho_hat_test']]
  row_fx_loss = [r'{:.1f}'.format(val1) for val1 in fx_rholoss_median.loc[fx_rholoss_median['rho_user']==rho,'%loss_red_test']]
  row_fx_lq = [r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) for val1,val2,val3 in zip(fx_CQT_median.loc[fx_CQT_median['rho_user']==rho,'loss_query_test'],\
                                                      fx_CQT_q3.loc[fx_CQT_q3['rho_user']==rho,'loss_query_test'],\
                                                      fx_CQT_q1.loc[fx_CQT_q1['rho_user']==rho,'loss_query_test'])]
  row_fx = row_fx + row_fx_CQT + row_fx_rho + row_fx_loss + row_fx_lq
  output_test.write("{:s}\\\\\n".format(" & ".join(row_fx)))

  row_gx_CQT = [r'({:.2f},{:.2f},{:.2f})'.format(val1,val2,val3) for val1,val2,val3 in zip(gx_CQT_median.loc[gx_CQT_median['rho_user']==rho,'queries_test'],\
                                                      gx_CQT_median.loc[gx_CQT_q3['rho_user']==rho,'corrected_test'],\
                                                      gx_CQT_median.loc[gx_CQT_q1['rho_user']==rho,'total_wrong_test'])]
  row_gx_rho = [r'{:.2f}'.format(val1) for val1 in gx_rholoss_median.loc[gx_rholoss_median['rho_user']==rho,'rho_hat_test']]
  row_gx_loss = [r'{:.1f}'.format(val1) for val1 in gx_rholoss_median.loc[gx_rholoss_median['rho_user']==rho,'%loss_red_test']]
  row_gx_lq = [r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) for val1,val2,val3 in zip(gx_CQT_median.loc[gx_CQT_median['rho_user']==rho,'loss_query_test'],\
                                                      gx_CQT_q3.loc[gx_CQT_q3['rho_user']==rho,'loss_query_test'],\
                                                      gx_CQT_q1.loc[gx_CQT_q1['rho_user']==rho,'loss_query_test'])]
  row_gx = row_gx + row_gx_CQT + row_gx_rho + row_gx_loss + row_gx_lq
  output_test.write("{:s}\\\\\n".format(" & ".join(row_gx)))
  output_test.write("\\cline{3-8}\n")


print(output_test.getvalue())

In [ ]:
# report_table_patch_only
report_table_patch_only_concat = pd.concat(report_table_patch_only)
report_table_patch_only_concat

In [ ]:
# report_table_patch
report_table_patch_concat = pd.concat(report_table_patch)
report_table_patch_concat

In [ ]:
p_value_col

In [ ]:
##%
# Boxplot (loss reduction in test set)
report_table_concat = pd.concat(report_table)
cols_table = ['p_value','rho_user','%reduction_test']
df_boxplot_table = pd.DataFrame(report_table_concat[cols_table])
df_boxplot_table['label'] = df_boxplot_table.shape[0]*['$f(x)$']
report_criteria_concat = pd.concat(report_criteria)
columns_crit = ['rho_user','%reduction_test']
df_boxplot_crit = pd.DataFrame(report_criteria_concat[columns_crit])
df_boxplot_crit['label'] = df_boxplot_crit.shape[0]*['$g(x)$']
# p-value median
p_value_col = df_boxplot_table['p_value'] #Add
p_value_by_row_index = df_boxplot_table['p_value'].groupby(df_boxplot_table.index)
p_value_median = p_value_by_row_index.median()
# Boxplot (jaccard index in test set)
columns_jac = ['rho_user','jaccard']
df_jaccard = pd.DataFrame(report_criteria_concat[columns_jac])
# Unfiltered Result dataframes
cols_fx = ['rho_user','%reduction_val','budget','%reduction_test']
results_fx = pd.DataFrame(report_table_concat[cols_fx])
cols_fxgx = ['rho_user','%reduction_test', 'jaccard']
results_fxgx = pd.concat([df_boxplot_table[cols_fxgx[:2]], df_boxplot_crit[cols_fxgx[1]], \
                          df_jaccard[cols_fxgx[2]]], axis=1)
# Filter experiments with p_value > 0.05
df_boxplot_table = df_boxplot_table.loc[df_boxplot_table['p_value'] <= 0.05]
df_boxplot_crit = df_boxplot_crit.loc[df_boxplot_table['p_value'] <= 0.05]
df_jaccard = df_jaccard.loc[df_boxplot_table['p_value'] <= 0.05]
# Boxplot with filtered values only
frames = [df_boxplot_table, df_boxplot_crit]
df = pd.concat(frames)

In [ ]:
# Avoid plotting when median(p_value)>0.5
for i in range(p_value_median.shape[0]):
    if p_value_median.iloc[i]>0.05:
      df.loc[df.index==i,'%reduction_test'] = np.nan
      df_jaccard.loc[df_jaccard.index==i, 'jaccard'] = np.nan

In [ ]:
# Dataframe for results f(x)
results_fx = results_fx.loc[p_value_col <= 0.05].copy()
results_fx_by_row_index = results_fx.groupby(results_fx.index)
fx_median = results_fx_by_row_index.median()
fx_q1 = results_fx_by_row_index.quantile(q=0.25)
fx_q3 = results_fx_by_row_index.quantile(q=0.75)
# Signaling function statistics (median(q1-q3)) LaTex
output_fx = io.StringIO()
numRows = fx_median.shape[0]
numCols = fx_median.shape[1]
output_fx.write("results_fx (\\rho|%reduction_val|sig_rate|%reduction_test|H0)\n")
output_fx.write("----------\n")
for i in range(numRows):
  row = [r'{:.2f}'.format(val1) if p_value_median[i]>0.05 and j==0 else r'{}' if p_value_median[i]>0.05 and j!=0\
         else r'{:.2f}'.format(val1) if (j==0) else r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) if (j==2)\
         else r'{:.1f}({:.1f}-{:.1f})'.format(val1,val2,val3) for val1,val2,val3,j in zip(fx_median.iloc[i],fx_q1.iloc[i],fx_q3.iloc[i],range(numCols))]
  output_fx.write("{{}} & {{}} & %s & {H0} \\\\\n".format(H0=r'$\surd$' if p_value_median[i]<=0.05 else r'$\times$')%(" & ".join(row)))
print(output_fx.getvalue())

In [ ]:
# Dataframe for comparison f(x)-g(x)
results_fxgx = results_fxgx.loc[p_value_col <= 0.05]
results_fxgx_by_row_index = results_fxgx.groupby(results_fxgx.index)
fxgx_median = results_fxgx_by_row_index.median()
fxgx_q1 = results_fxgx_by_row_index.quantile(q=0.25)
fxgx_q3 = results_fxgx_by_row_index.quantile(q=0.75)
# Baseline comparison statistics (median(q1-q3)) LaTex
output_fxgx = io.StringIO()
numRows = fxgx_median.shape[0]
numCols = fxgx_median.shape[1]
output_fxgx.write("results_fxgx (\\rho|%reduction_test(fx)|%reduction_test(fxgx)|Jaccard|H0\n")
output_fxgx.write("------------\n")
for i in range(numRows):
  row = [r'{:.2f}'.format(val1) if p_value_median[i]>0.05 and j==0 else r'{}' if p_value_median[i]>0.05 and j!=0\
         else r'{:.2f}'.format(val1) if (j==0) else r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3) if (j==3)\
         else r'{:.1f}({:.1f}-{:.1f})'.format(val1,val2,val3) for val1,val2,val3,j in zip(fxgx_median.iloc[i],fxgx_q1.iloc[i],fxgx_q3.iloc[i],range(numCols))]
  output_fxgx.write("{{}} & {{}} & %s & {H0} \\\\\n".format(H0=r'$\surd$' if p_value_median[i]<=0.05 else r'$\times$')%(" & ".join(row)))
print(output_fxgx.getvalue())

In [ ]:
#%%
# Save results in csv fomat
path_csv = "drive/My Drive/NIPS2020/results/socialadsnet/results_{clf}_yhat{yhat}_pca{pca}.csv".format(clf=clf, pca=applyPCA, yhat=addPredictions)
results = pd.concat([results_fx, results_fxgx, p_value_col.loc[p_value_col <= 0.05]], keys=['fx', 'fxgx', ''], axis=1).to_csv(path_csv, index=True, header=True)
# modified output
# Save results in tex fomat
L = [output_fx.getvalue(),output_fxgx.getvalue()]
path_txt = "drive/My Drive/NIPS2020/results/socialadsnet/results_{clf}_yhat{yhat}_pca{pca}.txt".format(clf=clf, pca=applyPCA, yhat=addPredictions)
txt = open(path_txt, "w") 
txt.writelines(L) 
txt.close() #to change file access modes 

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15, 5.1), constrained_layout=False, dpi=90)
pal = sns.color_palette('Paired')
sns.boxplot(x=df['rho_user'], y=df['%reduction_test'], hue='label', data=df, ax=ax[0], palette=pal)
ax[0].set_xlabel(r'budget $\rho$')
ax[0].set_ylabel(r'Loss reduction $r_{test}(\%)$')
ax[0].legend(loc='upper left')
pal = sns.color_palette('BuGn_r')
sns.boxplot(x=df_jaccard['rho_user'], y=df_jaccard['jaccard'], data=df_jaccard, ax=ax[1], palette=pal)
ax[1].set_xlabel(r'budget $\rho$')
ax[1].set_ylabel(r'Jaccard index $J$')
plt.tight_layout()
path_fig_fxgx = "drive/My Drive/NIPS2020/results/socialadsnet/fig_fxgx_{clf}_yhat{yhat}_pca{pca}.pdf".format(clf=clf, pca=applyPCA, yhat=addPredictions)
plt.savefig(path_fig_fxgx, bbox_inches='tight', facecolor='w')

In [ ]:
  #%%
# PLOT DECISION SURFACE
# ==================
# Plot test instances and decision surface
# ----------------------------------------------
# Visualising the Train set results
fig1 = plt.figure(figsize=(10,8),dpi=120)
ax1 = fig1.add_subplot(111)
X_set, y_set = X_train_surface_plot, y_train_surface_plot
y_set[y_set==0] = -1
if hasattr(classifier, "decision_function"):
    d_set = 1-y_set*classifier.decision_function(X_set)
else:
    pred_x = classifier.predict(X_set)
    pred_x[pred_x==0] = -1
    d_set = y_set*pred_x
xi_set = np.max([[np.zeros(d_set.size)],[d_set]],axis=0).ravel()

aranged_ages = np.arange(start = X_set[:, 0].min()-5, stop = X_set[:, 0].max()+5, step = 0.025)
aranged_salaries = np.arange(start = X_set[:, 1].min()-4000, stop = X_set[:, 1].max()+4000, step = 500)

X1, X2 = np.meshgrid(aranged_ages, aranged_salaries)
Z = classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape)
if hasattr(classifier, "decision_function"):
    d = classifier.decision_function(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape)
    ax1.contourf(X1, X2, np.where((np.abs(d)<1),np.abs(d),np.nan), alpha = 0.6, cmap='gist_gray', label='margin region')
    ax1.contour(X1, X2, d, levels=[-1, 0, 1], colors='black', linestyles='dashed')
    ax1.contourf(X1, X2, Z, alpha = 0.3, cmap = matplotlib.colors.ListedColormap(('red', 'blue')))
    x = X_set[xi_set>1,0]
    y = X_set[xi_set>1,1]
else:
    d = classifier.predict_proba(np.array([X1.ravel(), X2.ravel()]).T)[:, 1].reshape(X1.shape)-0.5
    ax1.contourf(X1, X2, np.where((np.abs(d)<0.25),np.abs(d),np.nan), alpha = 0.6, cmap='gist_gray', label='margin region')
    ax1.contour(X1, X2, d, levels=[-0.25, 0.0, 0.25], colors='black', linestyles='dashed')
    ax1.contourf(X1, X2, Z, alpha = 0.3, cmap = matplotlib.colors.ListedColormap(('red', 'blue')))
    x = X_set[xi_set==0,0]
    y = X_set[xi_set==0,1]
dots = ['red','blue']
for i, j in enumerate(np.unique(y_set)):
    ax1.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                color = dots[i], label = '{}'.format('no purchase' if i==0 else 'purchase'),alpha=0.7, s=65, linewidths=5)
# Plot slack variables magnitudes
ax1.scatter(x, y, s=150, facecolors='none', edgecolors='g', label='misclassified', linewidths=5)

ax1.set_xlabel('Age', fontsize=40)
ax1.set_ylabel('Salary', fontsize=40)
ax1.set_title('Training set', fontsize=40)
ax1.legend(loc='lower left', framealpha=0.5, prop={'size': 30}, labelspacing=0.0)
formatter0 = matplotlib.ticker.EngFormatter()
ax1.yaxis.set_major_formatter(formatter0)
X1_min, X1_max = X1.min(), X1.max()
X2_min, X2_max = X2.min(), X2.max()
ax1.set_xlim(X1_min, X1_max)
ax1.set_ylim(X2_min, X2_max)
plt.tight_layout()
path_fig_fx_train = "drive/My Drive/NIPS2020/results/socialadsnet/fig_fx_train_{clf}_yhat{yhat}_pca{pca}.svg".format(clf=clf, pca=applyPCA, yhat=addPredictions)
plt.savefig(path_fig_fx_train, bbox_inches='tight', facecolor='w')

In [ ]:
#%%
# PLOT DECISION SURFACE
# ==================
# Plot test instances and decision surface
# ----------------------------------------------
# Visualising the Train set results
fig1 = plt.figure(figsize=(8,8),dpi=120)
ax1 = fig1.add_subplot(111)
X_set, y_set = X_val_surface_plot, y_val_surface_plot
y_set[y_set==0] = -1
if hasattr(classifier, "decision_function"):
    d_set = 1-y_set*classifier.decision_function(X_set)
else:
    pred_x = classifier.predict(X_set)
    pred_x[pred_x==0] = -1
    d_set = y_set*pred_x
xi_set = np.max([[np.zeros(d_set.size)],[d_set]],axis=0).ravel()
if hasattr(classifier, "decision_function"):
    d = classifier.decision_function(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape)
    ax1.contourf(X1, X2, np.where((np.abs(d)<1),np.abs(d),np.nan), alpha = 0.6, cmap='gist_gray', label='margin region')
    ax1.contour(X1, X2, d, levels=[-1,0,1], colors='black', linestyles='dashed')
    ax1.contourf(X1, X2, Z, alpha = 0.3, cmap = matplotlib.colors.ListedColormap(('red', 'blue')))
    x = X_set[xi_set>1,0]
    y = X_set[xi_set>1,1]
else:
    d = classifier.predict_proba(np.array([X1.ravel(), X2.ravel()]).T)[:, 1].reshape(X1.shape)-0.5
    ax1.contourf(X1, X2, np.where((np.abs(d)<0.25),np.abs(d),np.nan), alpha = 0.6, cmap='gist_gray', label='margin region')
    ax1.contour(X1, X2, d, levels=[-0.25, 0, 0.25], colors='black', linestyles='dashed')
    ax1.contourf(X1, X2, Z, alpha = 0.3, cmap = matplotlib.colors.ListedColormap(('red', 'blue')))
    x = X_set[xi_set==0,0]
    y = X_set[xi_set==0,1]

dots = ['red','blue']
for i, j in enumerate(np.unique(y_set)):
    ax1.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                color = dots[i], label = '{}'.format('no purchase' if i==0 else 'purchase'),alpha=0.7,s=150, marker='*', linewidths=5)
# Plot slack variables magnitudes
ax1.scatter(x, y, s=150, facecolors='none', edgecolors='g', label='misclassified', linewidths=5)
 
ax1.set_xlabel('Age',fontsize=40)
ax1.set_yticks([])
ax1.set_title('Validation set',fontsize=40)
ax1.legend(loc='lower left', framealpha=0.5, prop={'size': 30},labelspacing=0.0)
ax1.set_xlim(X1_min, X1_max)
ax1.set_ylim(X2_min, X2_max)
plt.tight_layout()
path_fig_fx_val = "drive/My Drive/NIPS2020/results/socialadsnet/fig_fx_val_{clf}_yhat{yhat}_pca{pca}.svg".format(clf=clf, pca=applyPCA, yhat=addPredictions)
plt.savefig(path_fig_fx_val, bbox_inches='tight', facecolor='w')

In [ ]:
t = np.stack((X1.ravel(), X2.ravel()), axis=1)
t_scaled = scaler_surface_plot.transform(t)
f,v = exp_surface_plot.gpr.predict(t_scaled)
f = f.reshape(X1.shape)
# PLOT DECISION SURFACE
# ==================
# Plot test instances and decision surface
# ----------------------------------------------
# Visualising the Test set results
fig1 = plt.figure(figsize=(10,8),dpi=120)
ax1 = fig1.add_subplot(111)
X_set, y_set = X_test_surface_plot, y_test_surface_plot
y_set[y_set==0] = -1
if hasattr(classifier, "decision_function"):
    d_set = 1-y_set*classifier.decision_function(X_set)
else:
    pred_x = classifier.predict(X_set)
    pred_x[pred_x==0] = -1
    d_set = y_set*pred_x
xi_set = np.max([[np.zeros(d_set.size)],[d_set]],axis=0).ravel()
if hasattr(classifier, "decision_function"):
    d = classifier.decision_function(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape)
    ax1.contour(X1, X2, d, levels=[0], colors='black', linestyles='dashed')
    ax1.contourf(X1, X2, Z, alpha = 0.3, cmap = matplotlib.colors.ListedColormap(('red', 'blue')))
    x = X_set[xi_set>1,0]
    y = X_set[xi_set>1,1]
else:
    d = classifier.predict_proba(np.array([X1.ravel(), X2.ravel()]).T)[:, 1].reshape(X1.shape)
    ax1.contour(X1, X2, d, levels=[0.5], colors='black', linestyles='dashed')
    ax1.contourf(X1, X2, Z, alpha = 0.3, cmap = matplotlib.colors.ListedColormap(('red', 'blue')))
    x = X_set[xi_set==0,0]
    y = X_set[xi_set==0,1]

f_set = table_surface_plot['eta'].to_numpy()[::-1]
f_set = np.unique(np.around(np.append(f_set, f.max()), decimals=2))
cs = ax1.contourf(X1, X2, f, f_set, origin='upper', cmap='gray', alpha=0.5)
ax1.contour(X1, X2, f, f_set, colors='black')
cbar = fig1.colorbar(cs, pad=0.0, shrink=0.80)
cbar.ax.set_title(r'$f$', fontsize=40, loc='left')
cbar.set_label(r'$\eta$', labelpad=-10, y=1.10, rotation=0, fontsize=30)
dots = ['red','blue']
for i, j in enumerate(np.unique(y_set)):
    ax1.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                color = dots[i], label = '{}'.format('no purchase' if i==0 else 'purchase'), alpha=0.7, s=150,marker='x', linewidth=5)
# Plot slack variables magnitudes
ax1.scatter(x, y, s=150, facecolors='none', edgecolors='g', label='misclassified', linewidths=5)

ax1.set_xlabel('Age',fontsize=40)
ax1.set_yticks([])
ax1.set_title('Test set',fontsize=40)
ax1.legend(loc='lower left', framealpha=0.5, prop={'size': 30}, labelspacing=0.0)
ax1.set_xlim(X1_min, X1_max)
ax1.set_ylim(X2_min, X2_max)
plt.tight_layout()
path_fig_fx_test = "drive/My Drive/NIPS2020/results/socialadsnet/fig_fx_test_{clf}_yhat{yhat}_pca{pca}.svg".format(clf=clf, pca=applyPCA, yhat=addPredictions)
plt.savefig(path_fig_fx_test, bbox_inches='tight', facecolor='w')

In [ ]:
report_table_concat = pd.concat(report_table)
table_by_row_index = report_table_concat.groupby(report_table_concat.index)
report_table_mean = table_by_row_index.mean()
report_table_std = table_by_row_index.std()
report_table_median = table_by_row_index.median()
report_table_q1 = table_by_row_index.quantile(q=0.25)
report_table_q3 = table_by_row_index.quantile(q=0.75)

report_criteria_concat = pd.concat(report_criteria)
table_by_row_index = report_criteria_concat.groupby(report_criteria_concat.index)
report_criteria_mean = table_by_row_index.mean()
report_criteria_std = table_by_row_index.std()
report_criteria_median = table_by_row_index.median()
report_criteria_q1 = table_by_row_index.quantile(q=0.25)
report_criteria_q3 = table_by_row_index.quantile(q=0.75)

In [ ]:
##%
# Signaling function statistics (median(q1-q3))
output = io.StringIO()
columns = ['%reduction_val','budget','%reduction_test']
df1 = report_table_median[columns]
df2 = report_table_q1[columns]
df3 = report_table_q3[columns]
pval = report_table_mean['p_value']
rho = report_table_mean['rho_user']
assert(df1.shape[0]==df2.shape[0])
numRows = df1.shape[0]
for i in range(numRows):
  row = [r'{:.1f}({:.1f}-{:.1f})'.format(val1,val2,val3) if pval[i]<=0.05 and j!=1 else r'{:.2f}({:.2f}-{:.2f})'.format(val1,val2,val3)\
       if pval[i]<=0.05 and j==1 else r'{}' for val1,val2,val3,j in zip(df1.iloc[i],df2.iloc[i],df3.iloc[i],range(len(columns)))]
  output.write("{{}} & {{}} & {rho:.2f} & %s & {H0} \\\\\n".format(rho=rho[i], H0=r'$\surd$' if pval[i]<=0.05 else r'$\times$')%(" & ".join(row)))
print(output.getvalue())

In [ ]:
##%
# Signaling function statistics (mean+/-std)
output = io.StringIO()
columns = ['%reduction_val','budget','%reduction_test']
df1 = report_table_mean[columns]
df2 = report_table_std[columns]
pval = report_table_mean['p_value']
rho = report_table_mean['rho_user']
assert(df1.shape[0]==df2.shape[0])
numRows = df1.shape[0]
for i in range(numRows):
  row = [r'{:.1f}$\pm${:.1f}'.format(val1,val2) if pval[i]<=0.05 and j!=1 else r'{:.2f}$\pm${:.2f}'.format(val1,val2)\
       if pval[i]<=0.05 and j==1 else r'{}' for val1,val2,j in zip(df1.iloc[i],df2.iloc[i],range(len(columns)))]
  output.write("{{}} & {{}} & {rho:.2f} & %s & {H0} \\\\\n".format(rho=rho[i], H0=r'$\surd$' if pval[i]<=0.05 else r'$\times$')%(" & ".join(row)))
print(output.getvalue())

In [ ]:
##%
# Comparison statistics (median(q1-q3))
output = io.StringIO()
shared = ['%reduction_test']
df1 = report_table_median[shared]
df2 = report_table_q1[shared]
df3 = report_table_q3[shared]
shared.append('jaccard')
df4 = report_criteria_median[shared]
df5 = report_criteria_q1[shared]
df6 = report_criteria_q3[shared]
pval = report_table_mean['p_value']
rho = report_table_mean['rho_user']
assert(df1.shape[0]==df3.shape[0])
numRows = df1.shape[0]
for i in range(numRows):
  row_table = [r'{:.1f}({:.1f}-{:.1f})'.format(val1,val2,val3) if pval[i]<=0.05 else r'{}' for val1,val2,val3 in zip(df1.iloc[i],df2.iloc[i],df3.iloc[i])]
  row_criteria = [r'{:.1f}({:.1f}-{:.1f})'.format(val4,val5,val6) if pval[i]<=0.05 and j==0 else r'{:.2f}({:.2f}-{:.2f})'.format(val4,val5,val6)\
                  if pval[i]<=0.05 and j==1 else r'{}' for val4,val5,val6,j in zip(df4.iloc[i],df5.iloc[i],df6.iloc[i],range(len(columns)))]
  # row =  []
  output.write("{{}} & {{}} & {rho:.2f} & %s & %s & {H0} \\\\\n".format(rho=rho[i], \
                H0=r'$\surd$' if pval[i]<=0.05 else r'$\times$')%(" & ".join(row_table), " & ".join(row_criteria)))
print(output.getvalue())

In [ ]:
##%
# Comparison statistics (mean+/-std)
output = io.StringIO()
shared = ['%reduction_test']
df1 = report_table_mean[shared]
df2 = report_table_std[shared]
shared.append('jaccard')
df3 = report_criteria_mean[shared]
df4 = report_criteria_std[shared]
pval = report_table_mean['p_value']
rho = report_table_mean['rho_user']
assert(df1.shape[0]==df3.shape[0])
numRows = df1.shape[0]
for i in range(numRows):
  row_table = [r'{:.1f}$\pm${:.1f}'.format(val1,val2) if pval[i]<=0.05 else r'{}' for val1,val2 in zip(df1.iloc[i],df2.iloc[i])]
  row_criteria = [r'{:.1f}$\pm${:.1f}'.format(val3,val4) if pval[i]<=0.05 and j==0 else r'{:.2f}$\pm${:.2f}'.format(val3,val4)\
                  if pval[i]<=0.05 and j==1 else r'{}' for val3,val4,j in zip(df3.iloc[i],df4.iloc[i],range(len(columns)))]
  # row =  []
  output.write("{{}} & {{}} & {rho:.2f} & %s & %s & {H0} \\\\\n".format(rho=rho[i], \
                H0=r'$\surd$' if pval[i]<=0.05 else r'$\times$')%(" & ".join(row_table), " & ".join(row_criteria)))
print(output.getvalue())

In [ ]:
# data to plot
means_table = report_table_mean['%reduction_test'].to_numpy()
means_table[pval>0.05]=0
std_table = report_table_std['%reduction_test'].to_numpy()
std_table[pval>0.05]=0
means_crit = report_criteria_mean['%reduction_test'].to_numpy()
means_crit[pval>0.05]=0
std_crit = report_criteria_std['%reduction_test'].to_numpy()
std_crit[pval>0.05]=0
n_groups = means_table.size

# create plot
fig, ax = plt.subplots(figsize=(5.4, 5.1), constrained_layout=False, dpi=120)
index = np.arange(n_groups)
bar_width = 0.35
opacity = 0.6

rects1 = plt.bar(index, means_table, bar_width, yerr=std_table, capsize=5,
alpha=opacity, color='b', label=r'$f(x)>\eta$')

rects2 = plt.bar(index + bar_width, means_crit, bar_width, yerr=std_crit, capsize=5,
alpha=opacity, color='g', label=r'$g(x)>\theta$')

plt.xlabel(r'budget $\rho$')
plt.ylabel(r'loss reduction $r_{test}(\%)$')
# plt.title('Scores by person')
plt.xticks(index + bar_width/2, (['{}'.format(i) for i in report_table_mean['rho_user'].to_numpy()]))
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
0.37+0.37-0.9999